In [19]:
import sqlite3
con = sqlite3.connect('coin.db')
cursorObj = con.cursor()
cursorObj.execute("CREATE TABLE IF NOT EXISTS coin(id INTEGER PRIMARY KEY, symbol TEXT, amount INTEGER, price REAL)")
con.commit()

In [20]:
import pandas as pd

In [16]:
cursorObj.execute("SELECT * FROM coin")
coins = cursorObj.fetchall()
print(coins)

[(1, 'BTC', 2, 3250.0), (2, 'BNB', 5, 10.0), (4, 'ETH', 1, 140.0), (5, 'XMR', 5, 45.0), (6, 'XRP', 5, 0.2), (7, '', '', '')]


In [7]:
import requests
import json

In [9]:
api_request = requests.get("https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest?start=1&limit=300&convert=USD&CMC_PRO_API_KEY=c6e4c7e0-6413-44dd-85b2-9c145a762d4c")
api = json.loads(api_request.content)


In [50]:
print(api["data"][0]["quote"])


{'USD': {'price': 31241.892224520365, 'volume_24h': 67564480207.633865, 'percent_change_1h': -1.14939104, 'percent_change_24h': -1.56937486, 'percent_change_7d': -10.12596139, 'percent_change_30d': 18.54554912, 'market_cap': 581490281382.9453, 'last_updated': '2021-01-28T07:54:02.000Z'}}


In [34]:
table = pd.DataFrame(api["data"][1]["quote"],columns=api["data"][1]["quote"])

In [35]:
table.append(api["data"][2]["quote"],ignore_index=True)

,USD
0,2021-01-28T07:54:02.000Z
1,1.47493e+11
2,-0.453885
3,-0.487348
4,82.1719
5,-1.78546
6,1288.9
7,4.04889e+10
8,"{'price': 1.00098823453909, 'volume_24h': 1116..."


In [39]:
table = pd.DataFrame(api["data"])

In [48]:
api["data"][0]["quote"]["USD"]["price"]

31241.892224520365

In [67]:
con = sqlite3.connect('coin.db')
cursorObj = con.cursor()
cursorObj.execute("DROP TABLE IF EXISTS coin_data;")
cursorObj.execute("CREATE TABLE IF NOT EXISTS coin_data(id INTEGER PRIMARY KEY, symbol TEXT, price REAL, market_cap REAL, volume_24h REAL)")
con.commit()

for i in range(0, 300):
    symbol = api["data"][i]["symbol"]
    current_price = api["data"][i]["quote"]["USD"]["price"]
    market_cap = api["data"][i]["quote"]["USD"]["market_cap"]
    volume_24h = api["data"][i]["quote"]["USD"]["volume_24h"]
    cursorObj.execute("INSERT INTO coin_data(symbol, price, market_cap, volume_24h) VALUES(?, ?, ?, ?)", (symbol, current_price, market_cap, volume_24h))
    con.commit()
                                       
cursorObj.close()
con.close()                  

In [74]:
con = sqlite3.connect('coin.db')
cursorObj = con.cursor()
cursorObj.execute("SELECT * FROM coin_data")
coins = cursorObj.fetchall()
pd_frame = pd.DataFrame(coins)
pd_frame

,0,1,2,3,4
0,1,BTC,31241.892225,5.814903e+11,6.756448e+10
1,2,ETH,1288.895004,1.474932e+11,4.048891e+10
2,3,USDT,1.000988,2.542699e+10,1.116697e+11
3,4,DOT,16.463206,1.497170e+10,3.209655e+09
4,5,XRP,0.257028,1.167010e+10,2.979259e+09
...,...,...,...,...,...
295,296,BTU,0.517787,4.660830e+07,4.918038e+04
296,297,SNL,0.068463,4.646351e+07,8.021214e+05
297,298,LON,5.755002,4.524856e+07,7.355223e+07
298,299,CENNZ,0.037148,4.457778e+07,1.331886e+06
